# Day 9

## Part I

实现一个简单的虚拟机。首先定义一些帮助的数据结构，命令有三种，用下面的枚举类型体现：

In [1]:
from enum import Enum

Operation = Enum('Instruction', ('acc', 'jmp', 'nop'))

然后定义一个类来实现指令，在Rust中，可以全部在枚举中实现，Python只能用类：

In [2]:
class Instruction(object):
    def __init__(self, code_line: str):
        segs = code_line.rstrip().split()
        if segs[0] == 'acc':
            self.op = Operation.acc
        elif segs[0] == 'jmp':
            self.op = Operation.jmp
        elif segs[0] == 'nop':
            self.op = Operation.nop
        self.arg = int(segs[1])
        self.run_times = 0

虚拟游戏机的类，相当简单，ax是累加寄存器，ip是指令寄存器，codes是装载了所有指令的内存区域，就是一段列表。run方法执行一条指令，处理相应的ax和ip，还将该条指令的运行次数加1。run_to_repeatation用来完成执行到第一条重复指令的逻辑，也就是第一部分的问题

In [3]:
class GamePad(object):
    def __init__(self, code_file: str):
        self.ax = 0
        self.ip = 0
        with open(code_file) as fn:
            self.codes = [Instruction(line) for line in fn.readlines()]
    def run(self):
        inst = self.codes[self.ip]
        if inst.op == Operation.acc:
            self.ax += inst.arg
            self.ip += 1
        elif inst.op == Operation.nop:
            self.ip += 1
        elif inst.op == Operation.jmp:
            self.ip += inst.arg
        inst.run_times += 1
    def run_to_repeatation(self):
        while self.codes[self.ip].run_times == 0:
            self.run()

定义函数获取第一部分的答案：

In [4]:
def part1_solution(pad: GamePad) -> int:
    pad.run_to_repeatation()
    return pad.ax

单元测试

In [5]:
testcase = GamePad('testcase1.txt')
assert(part1_solution(testcase) == 5)

第一部分的结果：

In [6]:
pad = GamePad('input.txt')
part1_solution(pad)

1654

## Part II

第二部分引入了新的状态，因此我们需要重构GamePad类，同时引入一个新的枚举类型表示当前语句的执行状态，共有三种：正常运行normal；结束terminated；运行到重复指令repeated。同时改变run方法的签名，返回一个元组，包括状态、执行指令的ip和被执行的指令本身：

In [7]:
from typing import Tuple

Status = Enum('Status', ('normal', 'terminated', 'repeated'))

class GamePad(object):
    def __init__(self, code_file: str):
        self.ax = 0
        self.ip = 0
        with open(code_file) as fn:
            self.codes = [Instruction(line) for line in fn.readlines()]
    def run(self) -> Tuple[Status, int, Instruction]:
        if self.ip >= len(self.codes):
            return Status.terminated, self.ip, None
        inst = self.codes[self.ip]
        if inst.run_times >= 1:
            return Status.repeated, self.ip, inst
        rip = self.ip
        if inst.op == Operation.acc:
            self.ax += inst.arg
            self.ip += 1
        elif inst.op == Operation.nop:
            self.ip += 1
        elif inst.op == Operation.jmp:
            self.ip += inst.arg
        inst.run_times += 1
        return Status.normal, rip, inst

下面需要解决第二部分问题，首先我们需要记录下在指令发生重复之前，所有的jmp和nop指令，此处会将这些指令保存到一个列表中，然后遍历这个列表，每次将相应的nop改为jmp或将jmp改为nop，尝试执行，直至找到能够获得terminated状态为止，最后返回ax的值：

In [8]:
from copy import deepcopy

def part2_solution(pad: GamePad) -> int:
    instructions = []
    pad_clone = deepcopy(pad)
    ret = pad_clone.run()
    while ret[0] == Status.normal:
        if ret[2].op == Operation.jmp or ret[2].op == Operation.nop:
            instructions.append((ret[1], ret[2]))
        ret = pad_clone.run()
    for ip, inst in instructions:
        pad_clone = deepcopy(pad)
        pad_clone.codes[ip].op = Operation.nop if inst.op == Operation.jmp else Operation.jmp
        while True:
            ret = pad_clone.run()
            if ret[0] == Status.repeated:
                break
            if ret[0] == Status.terminated:
                return pad_clone.ax
    return None

单元测试

In [9]:
testcase = GamePad('testcase1.txt')
assert(part2_solution(testcase) == 8)

第二部分的结果：

In [10]:
pad = GamePad('input.txt')
part2_solution(pad)

833

同时，第一部分可以使用新的GamePad类来重新计算结果：

In [11]:
def part1_solution(pad: GamePad) -> int:
    while True:
        ret = pad.run()
        if ret[0] == Status.terminated or ret[0] == Status.repeated:
            return pad.ax

结果一致：

In [12]:
pad = GamePad('input.txt')
part1_solution(pad)

1654